In [49]:
# library import
import rpy2
import rpy2.robjects as robjects
import pandas as pd
import yaml
import requests
import parameters

In [77]:
# import R packages into python
"""
Load the R packages that will be used, such as: 
        - base
        - utils
        - nls.multstart
"""
from rpy2.robjects.packages import importr

r_base = importr('base') 
r_utils = importr('utils')
r_nlme = importr('nls')

PackageNotInstalledError: The R package "nls" is not installed.

In [ ]:
# import parameters data
"""
Import the parameters of hospitalization and mortality by state using data from the CoronaCidades
"""

data_parameters = pd.read_csv("http://datasource.coronacidades.org/br/states/parameters")
data_parameters

In [ ]:
# import farolcovid data
"""
Imports FarolCovid data related to:
- Updated cases and deaths
- Notification rate and most recent estimated active cases
- Most recent contagion rate (Rt)
- Latest number of sick and ICU beds from CNES
"""

data_farol = pd.read_csv("http://datasource.coronacidades.org/br/states/farolcovid/main").set_index("state_num_id")
df_farol = data_farol[["state_id", "last_updated_cases", "confirmed_cases", "deaths", 
          "notification_rate", "active_cases", "rt_most_likely", 
          "number_beds", "number_icu_beds", "last_updated_number_beds"]]
df_farol

In [ ]:
# parameters config import
"""
Import the disease progression times fixed parameters from the literature
"""
url = "https://raw.githubusercontent.com/ImpulsoGov/farolcovid/stable/src/configs/config.yaml"
config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)
config = config["br"]["seir_parameters"]
config


In [78]:
# data selection for the model and parametrs calculation
"""
The parametrs was calculed using the original seir.py scrip
"""

frac_severe_to_critical = data_parameters["i3_percentage"] / (
        data_parameters["i2_percentage"] + data_parameters["i3_percentage"])
    
p1 = 1 - (data_parameters["i1_percentage"]) / (config["mild_duration"])

p2 = frac_severe_to_critical / config["severe_duration"]

x = (
    0.9 
    * (1 / config["mild_duration"]) 
    * (config["severe_duration"]) 
    * (1 / config["critical_duration"])
    )

y = (
        p1 * (1 / config["critical_duration"])
        + p1 * p2
    )

I1 = df_farol["confirmed_cases"] * data_parameters["i1_percentage"]
I2 = df_farol["confirmed_cases"] * data_parameters["i2_percentage"]
I3 = df_farol["confirmed_cases"] * data_parameters["i3_percentage"]

beta1 = (
    0.9 
    * (1 / config["mild_duration"]) 
    * (config["severe_duration"]) * (1 / config["critical_duration"]))


beta3 = 0.1 * (x / y) * (config["severe_duration"]) / (1 / config["critical_duration"])

beta2 = beta3

In [79]:
# non-linear regression model for I1, I2 e I3 population initial states
"""
The non-linear regression model will use the parameters:

       Population parameters:
              - I1: infected mild
              - I2: infected severe
              - I3: infected critical

The Schumacher and Hal model is through the nls function from the nlme R package

The model formula is I1 ~ beta1 * (I2^beta2) * (I3^beta3)
"""
nlr_model = robjects.r(nls.multstart::nls_multstart(I1~beta1*(I2^beta2)*(I3^beta3)))



SyntaxError: invalid syntax (<ipython-input-79-8c1b9f586ca7>, line 14)

In [80]:
# model values

summary = robjects.r(sumary(nlr_model))

NameError: name 'sumary' is not defined